
**I referred to these links**
* https://developer.android.com/guide/topics/sensors/sensors_overview<br>
* https://www.kaggle.com/iamleonie/intro-to-indoor-location-navigation<br>
* https://github.com/danielmurray/adaptiv

In [ ]:
# a = "5cd56b5ae2acfd2d33b58544/F4/5cf22b433f260a00082c8dea.txt"
# a = "5cd56b5ae2acfd2d33b58544/F4/5cf22b453f260a00082c8dec.txt"
# a = "5cd56b5ae2acfd2d33b58544/F4/5cf22b440395f600088f6501.txt"
# a = "5cd56b5ae2acfd2d33b58544/F3/5cf22df20395f600088f653d.txt"
# a = "5cd56b5ae2acfd2d33b58544/F3/5cf22df20395f600088f653d.txt"

# a = "5cd56b77e2acfd2d33b5b22b/B1/5d09ed781a9498000952a05c.txt"
# a = "5cd56b77e2acfd2d33b5b22b/B1/5d09edca1a9498000952a062.txt"
# a = "5cd56b77e2acfd2d33b5b22b/F1/5d09f45324caf50008a4ef5b.txt"
# a = "5cd56b77e2acfd2d33b5b22b/F4/5d09f97024caf50008a4eff1.txt"
# a = "5cd56b77e2acfd2d33b5b22b/F2/5d09f5531a9498000952a11b.txt"
# a = "5cd56b77e2acfd2d33b5b22b/F3/5d09f8181a9498000952a151.txt"
a = "5cd56b77e2acfd2d33b5b22b/F5/5d09fa831a9498000952a1a4.txt"

# a = "5cd56865eb294480de7167b6/F2/5cfdd8fcffdc200008fddf30.txt"
# a = "5cd56b6ae2acfd2d33b59c90/F1/5cfdc042ffdc200008fdde0c.txt"
# a = "5cd56b70e2acfd2d33b5a44e/F3/5cf74718e36a480008125ed5.txt"

# a = "5a0546857ecc773753327266/B1/5e1581d4f4c3420006d52109.txt"
# a = "5cd969c839e2fc0b4afe7ff1/F5/5d0b3a544a58160008d75e26.txt"

# a = "5cd56b77e2acfd2d33b5b310/B1/5cf79e5ac12efb0008551804.txt"

b = "../input/indoor-location-navigation/train/" + a

In [ ]:
step_distance = 0.8
w_height = 1.7
m_trans = -5

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

from dataclasses import dataclass

import matplotlib.pyplot as plt # visualization
plt.rcParams.update({'font.size': 14})
import seaborn as sns # visualization

import warnings # Supress warnings 
warnings.filterwarnings('ignore')

from tqdm import tqdm

import json
import plotly.graph_objs as go
from PIL import Image

from datetime import datetime

In [ ]:
# copy from https://github.com/location-competition/indoor-location-competition-20/blob/master/io_f.py

@dataclass
class ReadData:
    acce: np.ndarray
    acce_uncali: np.ndarray
    gyro: np.ndarray
    gyro_uncali: np.ndarray
    magn: np.ndarray
    magn_uncali: np.ndarray
    ahrs: np.ndarray
    wifi: np.ndarray
    ibeacon: np.ndarray
    waypoint: np.ndarray


def read_data_file(data_filename):
    acce = []
    acce_uncali = []
    gyro = []
    gyro_uncali = []
    magn = []
    magn_uncali = []
    ahrs = []
    wifi = []
    ibeacon = []
    waypoint = []

    with open(data_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line_data in lines:
        line_data = line_data.strip()
        if not line_data or line_data[0] == '#':
            continue

        line_data = line_data.split('\t')

        if line_data[1] == 'TYPE_WAYPOINT':
            waypoint.append([int(line_data[0]), float(line_data[2]), float(line_data[3])])
            continue
       
        if line_data[1] == 'TYPE_ACCELEROMETER':
            acce.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue
        
        if line_data[1] == 'TYPE_ACCELEROMETER_UNCALIBRATED':
            acce_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue
        
        if line_data[1] == 'TYPE_GYROSCOPE':
            gyro.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE_UNCALIBRATED':
            gyro_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue
        
        if line_data[1] == 'TYPE_MAGNETIC_FIELD':
            magn.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD_UNCALIBRATED':
            magn_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ROTATION_VECTOR':
            ahrs.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_WIFI':
            sys_ts = line_data[0]
            ssid = line_data[2]
            bssid = line_data[3]
            rssi = line_data[4]
            lastseen_ts = line_data[6]
            wifi_data = [sys_ts, ssid, bssid, rssi, lastseen_ts]
            wifi.append(wifi_data)
            continue

        if line_data[1] == 'TYPE_BEACON':
            ts = line_data[0]
            uuid = line_data[2]
            major = line_data[3]
            minor = line_data[4]
            rssi = line_data[6]
            ibeacon_data = [ts, '_'.join([uuid, major, minor]), rssi]
            ibeacon.append(ibeacon_data)
            continue
        
    
    acce = np.array(acce)
    acce_uncali = np.array(acce_uncali)
    gyro = np.array(gyro)
    gyro_uncali = np.array(gyro_uncali)
    magn = np.array(magn)
    magn_uncali = np.array(magn_uncali)
    ahrs = np.array(ahrs)
    wifi = np.array(wifi)
    ibeacon = np.array(ibeacon)
    waypoint = np.array(waypoint)
    
    return ReadData(acce, acce_uncali, gyro, gyro_uncali, magn, magn_uncali, ahrs, wifi, ibeacon, waypoint)

sample_file = read_data_file(b)
#     "../input/indoor-location-navigation/train/5a0546857ecc773753327266/F2/5dccf516c04f060006e6e3c9.txt")

print('acce shape:', sample_file.acce.shape)
print('acce_uncali shape:', sample_file.acce_uncali.shape)
print('gyro shape:', sample_file.gyro.shape)
print('gyro_uncali shape:', sample_file.gyro_uncali.shape)
print('magn shape:', sample_file.magn.shape)
print('magn_uncali shape:',sample_file.magn_uncali.shape)
print('ahrs shape:', sample_file.ahrs.shape)
print('wifi shape:', sample_file.wifi.shape)
print('ibeacon shape:', sample_file.ibeacon.shape)
print('waypoint shape:', sample_file.waypoint.shape)


# Detect steps by acceleration

In [ ]:
sample_file.acce

* Combine three accelerations
* filtering

In [ ]:
mix_acce = np.sqrt(sample_file.acce[:,1:2]**2 + sample_file.acce[:,2:3]**2 + sample_file.acce[:,3:4]**2)
mix_acce = np.concatenate([sample_file.acce[:,0:1], mix_acce], 1)
mix_df = pd.DataFrame(mix_acce)
mix_df.columns = ["timestamp","acce"]
# mix_df

In [ ]:
from scipy.signal import butter, lfilter

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

order = 3
fs = 50.0  # sample rate, Hz
# fs = 100
# cutoff = 3.667  # desired cutoff frequency of the filter, Hz
cutoff = 3

filtered = butter_lowpass_filter(mix_df["acce"], cutoff, fs, order)
filtered

Actual linear movement distance（waypoint to waypoint）

In [ ]:
sample_file.waypoint

In [ ]:
distancex = np.diff(sample_file.waypoint[:,1])
distancey = np.diff(sample_file.waypoint[:,2])
distance = np.sqrt(distancex**2 + distancey**2)
distance = np.insert(distance, 0, 0)
distance = np.concatenate([sample_file.waypoint[:,0:1], distance.reshape(-1, 1)], 1)
distance

In [ ]:
dis_df = pd.DataFrame(distance)
dis_df.columns = ["timestamp","distance"]
dis_df

**Time from waypoint to waypoint**

In [ ]:
q = 0
d_time = []

for i in dis_df["timestamp"]:
    start_time = q
    end_time = i
    d_time.append(datetime.fromtimestamp(end_time/1000.0)-datetime.fromtimestamp(start_time/1000.0))
    q = i
d_time.pop(0)

for p in d_time:
    print(p)

**Filtered acceleration from waypoint to waypoint**

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 5))
plt.title("Filtered")
sns.lineplot(x=mix_df["timestamp"], y=mix_df["acce"], ax=ax, color='pink')
sns.lineplot(x=mix_df["timestamp"], y=filtered, ax=ax, color='cornflowerblue')

for i in dis_df["timestamp"]:
    ax.axvline(x=i, ymin=0, ymax=0.95,color='blue',ls='--',alpha=0.6)

plt.tight_layout()
plt.show()

**Detect steps**

In [ ]:
def peak_accel_threshold(data, timestamps, threshold):
# def peak_accel_threshold(data, timestamps):
    d_acc = []
    last_state = 'below'
    # below - less than threshold
    # above - above the threshold
    crest_troughs = 0
    crossings = []

    for i, datum in enumerate(data):
        
#         if len(d_acc) < 20:
#             d_acc.append(datum)
#             continue
#         else:
#             d_acc.append(datum)
#             d_acc.pop(0)
            
#         threshold = (max(d_acc)+min(d_acc)/2*1)
#         print(threshold)

        current_state = last_state
        if datum < threshold:
            current_state = 'below'
        elif datum > threshold:
            current_state = 'above'

        if current_state is not last_state:
            if current_state is 'above':
                crossing = [timestamps[i], threshold]
                crossings.append(crossing)
            else:
                crossing = [timestamps[i], threshold]
                crossings.append(crossing)

            crest_troughs += 1
        last_state = current_state

    return np.array(crossings)

In [ ]:
# threshold = 2.5
threshold = filtered.mean() * 1.1

crossings = peak_accel_threshold(filtered, mix_df["timestamp"], threshold)
# crossings = peak_accel_threshold(filtered, mix_df["timestamp"])

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 5))
plt.plot(mix_df["timestamp"], filtered, 'b-', color='cornflowerblue')
plt.plot(crossings.T[0], crossings.T[1], 'ro', linewidth=2)
plt.xlabel('Time [sec]')
for i in dis_df["timestamp"]:
    ax.axvline(x=i, ymin=0, ymax=0.95,color='blue',ls='--',alpha=0.6)
plt.grid()
plt.legend()
plt.show()


Two red dots are walking

In [ ]:
step_sum = len(crossings)/2
distance = w_height * 0.4 * step_sum
wp_distance_sum = dis_df["distance"].sum()
print(step_sum,distance,wp_distance_sum)

# Detect direction with magnetic field

In [ ]:
mag_df = pd.DataFrame(sample_file.magn)
mag_df.columns = ["timestamp","x","y","z"]
mag_df

In [ ]:
acce_df = pd.DataFrame(sample_file.acce)
acce_df.columns = ["timestamp","ax","ay","az"]
acce_df

In [ ]:
mag_df = pd.merge(mag_df,acce_df,on="timestamp")
mag_df

In [ ]:
mag_df.dropna()
mag_df

**The direction the surveyor is facing at each time stamp**

In [ ]:
time_di_list = []

for i in mag_df.iterrows():
    
    gx,gy,gz = i[1][1],i[1][2],i[1][3]
    ax,ay,az = i[1][4],i[1][5],i[1][6]
    
#     print(ax,ay,az)
    
    roll = math.atan2(ay,az)
    pitch = math.atan2(-1*ax , (ay * math.sin(roll) + az * math.cos(roll)))
    
    q =  m_trans - math.degrees(math.atan2(
        (gz*math.sin(roll)-gy*math.cos(roll)),(gx*math.cos(pitch) + gy*math.sin(roll)*math.sin(pitch) + gz*math.sin(pitch)*math.cos(roll))
    )) -90
    if q <= 0:
        q += 360
    time_di_list.append((i[1][0],q))

d_list = [x[1] for x in time_di_list]
# d_list

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 5))
plt.plot(mix_df["timestamp"], filtered,'b-', color='cornflowerblue')
plt.plot(crossings.T[0], crossings.T[1], 'ro', linewidth=2)
plt.xlabel('Time [sec]')
for i in dis_df["timestamp"]:
    ax.axvline(x=i, ymin=0, ymax=0.95,color='blue',ls='--',alpha=0.6)
ax2 = ax.twinx()
ax2.set_ylabel("digree")
sns.lineplot(x=mix_df["timestamp"], y=d_list[:len(mix_df["timestamp"])], ax=ax2, color='black')
plt.grid()
plt.legend()
plt.show()

**Direction when walking**

In [ ]:
steps = []
step_time = []
di_dict = dict(time_di_list)

for n,i in enumerate(crossings[:,:1]):
    if n % 2 == 1:
        continue
    direct_now = di_dict[i[0]]
    dx = math.sin(math.radians(direct_now))
    dy = math.cos(math.radians(direct_now))
    print(int(n/2+1),"歩目/x:",dx,"/y:",dy,"/角度：",direct_now)
    steps.append((i[0],dx,dy))
    step_time.append(i[0])


**Step-to-step time difference To the stride**

In [ ]:
step_dtime = np.diff(step_time)/1000
step_dtime = step_dtime.tolist()
step_dtime.insert(0,5)

**Waypoint-to-waypoint moving distance step number * stride**

In [ ]:
wp_time = dis_df["timestamp"].tolist()
wp_time.pop(0)

first_point = (sample_file.waypoint[0,1],sample_file.waypoint[0,2])
(step_x, step_y) = first_point
pred_px = [step_x]
pred_py = [step_y]

wp_idx = 0
print("WP:",round(sample_file.waypoint[0,1],3),round(sample_file.waypoint[0,2],3),sample_file.waypoint[0,0])
print("------------------")
for p,i in enumerate(steps):
    step_distance = 0
    if step_dtime[p] >= 1:
        step_distance = w_height*0.25
    elif step_dtime[p] >= 0.75:
        step_distance = w_height*0.3
    elif step_dtime[p] >= 0.5:
        step_distance = w_height*0.4
    elif step_dtime[p] >= 0.4:
        step_distance = w_height*0.5
    elif step_dtime[p] >= 0.3:
        step_distance = w_height*0.6
    else:
        step_distance = w_height*0.4
    
    step_x += i[1]*step_distance
    step_y += i[2]*step_distance
    
    if i[0] > wp_time[0]:
        print("PD:",round(step_x,3),round(step_y,3),i[0])
        pred_px.append(step_x)
        pred_py.append(step_y)
        wp_time.pop(0)
        wp_idx += 1
        print("WP:",round(sample_file.waypoint[wp_idx,1],3),round(sample_file.waypoint[wp_idx,2],3),sample_file.waypoint[wp_idx,0])
        print("------------------")
    if not wp_time:
        break
if wp_time:
    pred_px.append(step_x)
    pred_py.append(step_y)
    print("PD:",round(step_x,3),round(step_y,3),i[0])
    print("WP:",round(sample_file.waypoint[wp_idx+1,1],3),round(sample_file.waypoint[wp_idx+1,2],3),sample_file.waypoint[wp_idx+1,0])

In [ ]:
def visualize_trajectory(trajectory, floor_plan_filename, width_meter, height_meter, title=None, mode='lines + markers + text', show=False):
    """
    Copied from from https://github.com/location-competition/indoor-location-competition-20/blob/master/visualize_f.py

    """
    fig = go.Figure()

    # add trajectory
    size_list = [6] * trajectory.shape[0]
    size_list[0] = 10
    size_list[-1] = 10

    color_list = ['rgba(4, 174, 4, 0.5)'] * trajectory.shape[0]
    color_list[0] = 'rgba(12, 5, 235, 1)'
    color_list[-1] = 'rgba(235, 5, 5, 1)'

    position_count = {}
    text_list = []
    for i in range(trajectory.shape[0]):
        if str(trajectory[i]) in position_count:
            position_count[str(trajectory[i])] += 1
        else:
            position_count[str(trajectory[i])] = 0
        text_list.append('        ' * position_count[str(trajectory[i])] + f'{i}')
    text_list[0] = 'Start 0'
    text_list[-1] = f'End {trajectory.shape[0] - 1}'

    fig.add_trace(go.Scatter(x=pred_px, y=pred_py, name="pred"))
    
    fig.add_trace(
        go.Scattergl(
            x=trajectory[:, 0],
            y=trajectory[:, 1],
            mode=mode,
            marker=dict(size=size_list, color=color_list),
            line=dict(shape='linear', color='lightgrey', width=3, dash='dash'),
            text=text_list,
            textposition="top center",
            name='trajectory',
        ))

    # add floor plan
    floor_plan = Image.open(floor_plan_filename)
    fig.update_layout(images=[
        go.layout.Image(
            source=floor_plan,
            xref="x",
            yref="y",
            x=0,
            y=height_meter,
            sizex=width_meter,
            sizey=height_meter,
            sizing="contain",
            opacity=1,
            layer="below",
        )
    ])

    # configure
    fig.update_xaxes(autorange=False, range=[0, width_meter])
    fig.update_yaxes(autorange=False, range=[0, height_meter], scaleanchor="x", scaleratio=1)
    fig.update_layout(
        title=go.layout.Title(
            text=title or "No title.",
            xref="paper",
            x=0,
        ),
        autosize=True,
        width=800,
        height=  800 * height_meter / width_meter,
        template="plotly_white",
    )

    if show:
        fig.show()

    return fig

def visualize_train_trajectory(path):
    """
    Edited from 
    https://www.kaggle.com/ihelon/indoor-location-exploratory-data-analysis
    """
    _id, floor = path.split("/")[:2]
    
    train_floor_data = read_data_file(f"../input/indoor-location-navigation/train/{path}")
    with open(f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_info.json") as f:
        train_floor_info = json.load(f)

    return visualize_trajectory(
        train_floor_data.waypoint[:, 1:3], 
        f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_image.png",
        train_floor_info["map_info"]["width"], 
        train_floor_info["map_info"]["height"],
        f"Visualization of {path}"
    )
visualize_train_trajectory(a)